In [1]:
import numpy as np

#SEF imports
from stableemrifisher.utils import generate_PSD, padding, inner_product

#few imports
from few.trajectory.inspiral import EMRIInspiral #function to generate trajectories
from few.trajectory.ode.flux import KerrEccEqFlux #Adiabatic KerrEccEq base trajectory class

from few.waveform import (
    FastKerrEccentricEquatorialFlux,
    GenerateEMRIWaveform,
)
from few.utils.constants import YRSID_SI

#lisa-on-gpu import
from fastlisaresponse import ResponseWrapper  # Response function 
#LISAanalysistools imports
from lisatools.detector import ESAOrbits, EqualArmlengthOrbits #ESAOrbits correspond to esa-trailing-orbits.h5, EqualArmlengthOrbits are equalarmlength-orbits.h5
from lisatools.sensitivity import get_sensitivity, A1TDISens, E1TDISens, T1TDISens

use_gpu = False #False if your computer sucks (mine does)

if not use_gpu:
    
    import few
    
    #tune few configuration
    cfg_set = few.get_config_setter(reset=True)
    
    cfg_set.enable_backends("cpu")
    cfg_set.set_log_level("info");
else:
    pass #let the backend decide for itself.

/home/shubham/miniconda3/envs/SKeqfull/lib/python3.12/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
#fixed parameters
T_LISA = 0.1 #observation time, years. Should be 2.0 for actual runs.
dt = 10.0 #sampling interval, seconds
emri_kwargs = {"T":T_LISA, "dt": dt}

m1 = 1e6 #MBH mass in solar masses (source frame)
m2 = 10.0 #secondary mass in solar masses (source frame)
a = 0.9 #MBH spin (dimensionless)
p0 = 10.0 #init semi-latus rectum (M)
e0 = 0.1 #init eccentricity
x0 = 1.0 #inclination, must be = 1.0 for equatorial model

# initial phases
Phi_phi0 = 0.0 #azimuthal phase
Phi_theta0 = 0.0 #polar phase
Phi_r0 = 0.0 #radial phase

# define the extrinsic parameters
qK = np.pi / 3  # polar spin angle
phiK = np.pi / 4  # azimuthal viewing angle
qS = np.pi / 6  # polar sky angle
phiS = np.pi / 8  # azimuthal viewing angle
dist = 1.0  # distance in Gpc. We'll adjust this later to fix the SNR as SNR_fixed

In [6]:
#initialize waveform model
sum_kwargs = {
    "pad_output": True, # True if you want waveforms always of the length of the LISA observation window (pads by zero if time-series is smaller). Recommended to be True if passing to SEF for near-plunge sources.
    }

max_step_days = 10.0 #max trajectory step size in days. Smaller value leads to more stable trajectories, but takes longer to compute. Likely needs fiddling.

inspiral_kwargs = {
    "err":1e-11, #error tolerance in the DOPR integration routine. Default = 1e-11
    "max_step_size":max_step_days*24*60*60, #in seconds
}

waveform_model = GenerateEMRIWaveform(
            FastKerrEccentricEquatorialFlux, #the waveform class, see few.waveform.waveform.SuperKludgeWaveform
            sum_kwargs=sum_kwargs,
            inspiral_kwargs=inspiral_kwargs,
            return_list=False, #Must be False when interfacing with SEF: If ResponseWrapper provided, it automatically converts to lists. If GenerateEMRIWaveform supplied to SEF, it creates three equally-weighted copies as lists.
            )

#initialize LISA response model
tdi_gen ="1st generation"# "1st generation"#

order = 20  # interpolation order (should not change the result too much)
tdi_kwargs_esa = dict(
    orbits=EqualArmlengthOrbits(use_gpu=use_gpu),
    order=order, 
    tdi=tdi_gen, 
    tdi_chan="AET",
)

index_lambda = 8 #azimuthal sky location (phiS)
index_beta = 7 #polar sky location (qS)

# with longer signals we care less about this
t0 = 10000.0  # throw away on both ends when our orbital information is weird

waveform_response = ResponseWrapper(
                        waveform_gen=waveform_model, #the waveform model around which to wrap the Response
                        Tobs=T_LISA,
                        t0=t0,
                        dt=dt,
                        index_lambda=index_lambda,
                        index_beta=index_beta,
                        flip_hx=True,  # set to True if waveform is h+ - ihx (FEW is)
                        use_gpu=use_gpu,
                        is_ecliptic_latitude=False,  # False if using polar angle (theta)
                        remove_garbage="zero",  # removes the beginning of the signal that has bad information
                        **tdi_kwargs_esa,
                        )

In [7]:
#noise model
channels = [A1TDISens, E1TDISens, T1TDISens] #AET TDI channels. This should be the same as the channels used in ResponseWrapper init
noise_kwargs = [{"sens_fn": channel_i} for channel_i in channels] #Keyword argument for each channel to be supplied to get_sensitivity inside SEF

In [8]:
#generate waveform
wave_params = [m1, m2, a, p0, e0, x0, dist, qS, phiS, qK, phiK, Phi_phi0, Phi_theta0, Phi_r0]

wave = np.array(waveform_response(*wave_params, **emri_kwargs))

In [9]:
#generate the PSD function for the given noise model
PSD_funcs = generate_PSD(wave, dt = dt, 
                         noise_PSD = get_sensitivity, #noise model
                         channels = channels, #noise channels
                         noise_kwargs = noise_kwargs, #kwargs for noise model
                         use_gpu = use_gpu)

print(PSD_funcs[0].shape)

#calculate the inner product (SNR)
SNR = np.sqrt(inner_product(wave, wave, PSD_funcs, dt, use_gpu = use_gpu))
print("SNR: ", SNR)

(157790,)
SNR:  6.585203086645125
